<a href="https://colab.research.google.com/github/ColinPeppler/spring2020-meetings/blob/master/DeployModelToWeb/Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer		# Vectorize sentences
from sklearn.linear_model import LogisticRegression					  # Creates a Naive Bayes model
import pandas as pd												                    # Stores data in a table

In [2]:
# Retrieving data directly from a github repo
url = 'https://raw.githubusercontent.com/vt-ai-ml/fall2019-meetings/master/data/movie-pang02.csv'
data = pd.read_csv(url)

In [3]:
# A Vector Space Model where sentences are mapped to a vector space
vsm = TfidfVectorizer(stop_words='english')


# Get our data to fit our Logistic Regression model to
X = vsm.fit_transform(data['text'])
Y = data['class']
my_model = LogisticRegression()
my_model.fit(X, Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [4]:
# Transform these two samples into a vector
sample = ['I love this amazing movie', 'this movie is ass']
sample_vec = vsm.transform(sample)
result = my_model.predict(sample_vec)

print(sample[0] + ' is ' + result[0])
print(sample[1] + ' is ' + result[1])

I love this amazing movie is Pos
this movie is ass is Neg


In [5]:
# pip installs some modules we need that Colab doesn't come with already
!pip install flask-ngrok
!pip install flask-cors         

In [6]:
import flask                                  # web dev framework to set up an endpoint for our http reqquest
from flask_ngrok import run_with_ngrok        # gives our flask app a public url
from flask_cors import CORS                   # enables cross origin resource sharing

In [7]:
# start our flask app
app = flask.Flask(__name__)
CORS(app)
run_with_ngrok(app)

In [8]:
@app.route('/predict', methods=['POST'])
def predict():
	# Get our sentence from the POST request
	data = flask.request.get_json(force=True)
	sentence = [data['sentence']]			        # NOTE: vsm will only accept iterable objects, thus wrap in a list

	# Vectorize our sentence using VSM and predict using Naive Bayes model
	sentence_vec = vsm.transform(sentence)
	prediction = my_model.predict(sentence_vec)
	output = prediction[0]

	# Return our prediction as a JSON object
	json_output = flask.jsonify(str(output))
	return json_output

When you run the cell before, note the ngrok url that's provided. 

The output we're interested in is: `Running on http://2a60c9ff37b2.ngrok.io`. However, you will get a different url than this.


In [9]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2a60c9ff37b2.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


The github url provides an index.html file which is the code for our basic webpage. You can open the webpage by dragging the index.html file into the browser or running this in your terminal `open index.html` (make sure you do this in the directory where the index.html is stored).

**HINT**: Replace the \<url\> (Line 38) in the index.html with the ngrok url you copied from the last cell.

https://github.com/ColinPeppler/spring2020-meetings/blob/master/DeployModelToWeb/index.html

